In [2]:
import os
import pandas as pd

# Directories
folders = ['ciq_files', 'templates', 'logs']
for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)

# Create CIQ Excel files
df_airtel = pd.DataFrame({
    'Parameter': ['x1', 'x2', 'x3'],
    'Value': [10, 20, 30]
})
df_jio = pd.DataFrame({
    'Parameter': ['y1', 'y2', 'y3'],
    'Value': [40, 50, 60]
})

df_airtel.to_excel('ciq_files/airtel_config.xlsx', index=False)
df_jio.to_excel('ciq_files/jio_config.xlsx', index=False)

# Create Template files
with open('templates/airtel_template.txt', 'w') as f:
    f.write("@BaseStationA\nparam1, param2\n100, 200\n")
with open('templates/jio_template.txt', 'w') as f:
    f.write("@BaseStationB\nparamA, paramB\n300, 400\n")

# Create Log files
with open('logs/airtel_log.txt', 'w') as f:
    f.write("2025-05-11 12:00:00 [INFO] Base station configured successfully with parameters x1=10, x2=20.\n")
with open('logs/jio_log.txt', 'w') as f:
    f.write("2025-05-11 12:05:00 [INFO] Base station configured with parameters y1=40, y2=50.\n")

print("Sample files created!")


Sample files created!


In [8]:
# pip install pandas openpyxl sentence-transformers transformers langchain-community langchain-core

import os
import glob
import pandas as pd
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
import numpy as np

from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate

# 📁 Paths
CIQ_FOLDER = './ciq_files'
TEMPLATES_FOLDER = './templates'
LOGS_FOLDER = './logs'

# 🔧 Models
embedder = SentenceTransformer('all-MiniLM-L6-v2')
model_name = "gpt2"  # or use 'distilbert-base-uncased', 'bert-base-uncased', etc.

# Load the tokenizer and model directly (no authentication needed)
tokenizer = AutoTokenizer.from_pretrained(model_name)


llm = ChatOllama(model="llama3.1:8b")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

C:\Users\saip9\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\saip9\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

C:\Users\saip9\AppData\Local\Temp\ipykernel_17768\956159988.py:26: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3.1:8b")


In [16]:
!pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
  Using cached faiss_cpu-1.11.0-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
    --------------------------------------- 0.3/15.0 MB ? eta -:--:--
   -- ------------------------------------- 0.8/15.0 MB 2.6 MB/s eta 0:00:06
   --- ------------------------------------ 1.3/15.0 MB 2.9 MB/s eta 0:00:05
   ---- ----------------------------------- 1.6/15.0 MB 2.3 MB/s eta 0:00:06
   ------ --------------------------------- 2.4/15.0 MB 2.4 MB/s eta 0:00:06
   ------- -------------------------------- 2.9/15.0 MB 2.5 MB/s eta 0:00:05
   --------- ------------------------------ 3.7/15.0 MB 2.6 MB/s eta 0:00:05
   ----------- ---------------------------- 4.2/15.0 MB 2.6 MB/s eta 0:00:05
   ------------ --------------------------- 4.7/15.0 MB 2.6 MB/s eta 0:00:04
   ------------- -------------------------- 5.2/15.0 MB 2.6 MB/s eta 0:00:04
   ----------


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
import os, glob
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

# === Settings ===
CIQ_FOLDER = "ciq_files"
TEMPLATES_FOLDER = "templates"
LOGS_FOLDER = "logs"

# === Embedding model ===
embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# === Helper: Load and wrap docs ===
def load_docs(folder, doc_type):
    docs = []
    print(f"Loading {doc_type} docs from: {folder}")
    for file in glob.glob(f"{folder}/*"):
        print(f"  Reading: {file}")
        try:
            if file.endswith(".xlsx") and doc_type == "ciq":
                df = pd.read_excel(file, sheet_name=None)
                text = "\n".join(
                    f"Sheet: {sheet}\n{df[sheet].head(5).to_csv(index=False)}"
                    for sheet in df
                )
            else:
                with open(file, "r") as f:
                    text = f.read()
            docs.append(Document(page_content=text, metadata={"type": doc_type, "path": file}))
        except Exception as e:
            print(f"[!] Failed to read {file}: {e}")
    print(f"Loaded {len(docs)} {doc_type} docs\n")
    return docs


# === Load documents ===
ciq_docs = load_docs(CIQ_FOLDER, "ciq")
template_docs = load_docs(TEMPLATES_FOLDER, "template")
log_docs = load_docs(LOGS_FOLDER, "log")

# === Store each in separate FAISS DB ===
db_ciq = FAISS.from_documents(ciq_docs, embedder)
db_template = FAISS.from_documents(template_docs, embedder)
db_log = FAISS.from_documents(log_docs, embedder)

# === Routing Agent ===
router_llm = ChatOllama(model="llama3")

def route_db(query):
    prompt = PromptTemplate.from_template("""
Given the query below, classify it into one of the following categories:
- ciq: Related to Excel/CIQ network config
- template: Related to network configuration templates
- log: Related to logs or error/debug info

Query: "{query}"
Category:""")
    category = router_llm.invoke(prompt.format(query=query)).content.strip().lower()
    if "ciq" in category:
        return db_ciq
    elif "template" in category:
        return db_template
    elif "log" in category:
        return db_log
    else:
        return db_template  # fallback


Loading ciq docs from: ciq_files
  Reading: ciq_files\airtel_config.xlsx
  Reading: ciq_files\jio_config.xlsx
Loaded 2 ciq docs

Loading template docs from: templates
  Reading: templates\airtel_template.txt
  Reading: templates\jio_template.txt
Loaded 2 template docs

Loading log docs from: logs
  Reading: logs\airtel_log.txt
  Reading: logs\jio_log.txt
Loaded 2 log docs



In [31]:
!ollama pull llama3

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c... 100% ▕████████████████▏  110 B                         
pulling 3f8eb4da87fa... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [ ]:

# === Run the system ===
query = "give me jio_template"
selected_db = route_db(query)

# Retrieve docs from selected vector DB
results = selected_db.similarity_search(query, k=1)
context = results[0].page_content if results else "No relevant context found."

# === Final Prompt ===
final_prompt = PromptTemplate.from_template("""
You are an intelligent assistant. Use the context to answer the query.

Context:
{context}

User Query: {query}

Answer:""")
llm = Ollama(model="llama3.1:8b")
response = llm.invoke(final_prompt.format(context=context, query=query))

# === Output ===
print("\n✅ Selected source:", results[0].metadata["type"] if results else "None")
print("\n🧠 Response:\n", response)



✅ Selected source: log

🧠 Response:
 2025-05-11 12:05:00 [INFO] Base station configured with parameters y1=40, y2=50.


In [25]:
!ollama list


NAME                  ID              SIZE      MODIFIED     
llama3.1:8b           46e0c10c039e    4.9 GB    5 months ago    
qwen2.5-coder:1.5b    6d3abb8d2d53    986 MB    5 months ago    


In [4]:
!pip install pandas openpyxl sentence-transformers transformers langchain-community langchain-core

Defaulting to user installation because normal site-packages is not writeable
  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached torch-2.7.0-cp312-cp312-win_amd64.whl.metadata (29 kB)
  Using cached huggingface_hub-0.31.1-py3-none-any.whl.metadata (13 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
Using cached sentence_transformers-4.1.0-py3-none-any.whl (345 kB)
Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 20.6 MB/s eta 0:00:00
Using cached huggingface_hub-0.31.1-py3-none-any.whl (484 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
